In [1]:
import json
import urllib
import urllib.request
from urllib.parse import urlparse
import httplib2 as http  # external library
import pandas as pd
# from datetime import datetime, timezone
import os
import shutil
import schedule
import time
import datetime

C:\Users\User\AppData\Local\Temp\ipykernel_16400\1498473977.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
class ApiCall:
    def __init__(self, parent_dir):
        self.parent_dir = parent_dir
        os.chdir(parent_dir)
        # folder where data retrieve are stored
        self.folder = os.path.join(self.parent_dir, "classifier")
        if os.path.isdir(self.folder):
            # clear old data from folder before making api call
            shutil.rmtree(self.folder)
        os.makedirs(self.folder)

    def api_get_json(self, uri, path):
        """
        Makes request to retrive data from LTA DataMall.
        """
        # authentication parameters
        headers = {
            "AccountKey": "Qzo6MFd/SXWzO6fjzlmGvg==",
            "accept": "application/json",
        }  # this is by default

        # build query string & specify type of API call
        target = urlparse(uri + path)
        method = "GET"
        body = ""

        h = http.Http()  # get handle to http
        response, content = h.request(
            target.geturl(), method, body, headers)  # retrieve data

        # get data from content
        jsonObj = json.loads(content)  # dictonary
        data = jsonObj["value"]
        return data

    def download_speedband(self):
        """
        Retrieve speedband data from LTA DataMall. Location is split into start latitude & longitude and end latitude & longitude (of type float).
        Store as csv in assets folder.
        """
        uri = "http://datamall2.mytransport.sg"  # resource URL
        speed_path = "/ltaodataservice/TrafficSpeedBandsv2"
        data = self.api_get_json(uri, speed_path)
        df = pd.DataFrame(data)

        df['AvgSpeed'] = df['SpeedBand'].apply(
            lambda x: x * 10 - 4.5)

        df["Location"] = df["Location"].apply(lambda x: x.split(" "))
        df[
            ["StartLatitude", "StartLongitude", "EndLatitude", "EndLongitude"]
        ] = pd.DataFrame(df["Location"].to_list())
        df = df.astype(
            {
                "StartLatitude": "float",
                "StartLongitude": "float",
                "EndLatitude": "float",
                "EndLongitude": "float",
            }
        )
        # average latitude, longitude of stretch of road
        df["AvgLat"] = (df.StartLatitude + df.EndLatitude) / 2
        df["AvgLon"] = (df.StartLongitude + df.EndLongitude) / 2

        # Generate a unique filename with timestamp
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"speedbands_{timestamp}.csv"

        # Save file
        df.to_csv(os.path.join(self.folder, filename), index=False)

    def download_images(self):
        """
        Retrieves traffic images of 87 cameras from LTA DataMall.
        Store images in assets folder.
        """
        uri = "http://datamall2.mytransport.sg"  # resource URL
        img_path = "/ltaodataservice/Traffic-Imagesv2"
        data = self.api_get_json(uri, img_path)
        for i, dict in enumerate(data):
            link = dict["ImageLink"]
            filename = link.split("/")[5].split("?")[0]
            urllib.request.urlretrieve(
                link, os.path.join(self.folder, filename))
            
    def download_train_images(self):
        """
        Retrieves traffic images of 87 cameras from LTA DataMall.
        Store images in assets folder.
        """
        uri = "http://datamall2.mytransport.sg"  # resource URL
        img_path = "/ltaodataservice/Traffic-Imagesv2"
        data = self.api_get_json(uri, img_path)

        # Check if the folder exists, if not create it
        if not os.path.exists(self.folder):
            os.makedirs(self.folder)

        # Check for existing images in the folder
        existing_images = set(os.listdir(self.folder))

        for i, dict in enumerate(data):
            link = dict["ImageLink"]
            filename = link.split("/")[5].split("?")[0]

            # Check if the image already exists, if not download and save
            if filename not in existing_images:
                urllib.request.urlretrieve(
                    link, os.path.join(self.folder, filename))
                print(f"Downloaded {filename}")

    def download_incidents(self):
        """
        Retrieves traffic incidents data from LTA DataMall.
        Store as csv in assets folder.
        """
        uri = "http://datamall2.mytransport.sg"  # resource URL
        incident_path = "/ltaodataservice/TrafficIncidents"
        data = self.api_get_json(uri, incident_path)
        df = pd.DataFrame(data)

        # Generate a unique filename with timestamp
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"incidents_{timestamp}.csv"

        # Save file
        df.to_csv(os.path.join(self.folder, filename), index=False)

    def schedule_image_download(self, interval_minutes):
        """
        Schedule image download at specified intervals.
        """
        print(f"Scheduling image download every {interval_minutes} minutes...")
        schedule.every(interval_minutes).minutes.do(self.download_train_images)

        while True:
            schedule.run_pending()
            time.sleep(1)

    def clear_data(self):
        shutil.rmtree(self.folder)

In [4]:
dir = r"C:\Users\User\Desktop\DSO\traffic-prediction\backend"
api_call = ApiCall(dir)
while True:
    try:
        # Download incidents
        api_call.download_incidents()

        # Download speedbands
        api_call.download_speedband()

        # Download images
        api_call.download_train_images()

        print("Downloads completed successfully.")

        # Sleep for some time before next iteration
        time.sleep(60)  # Sleep for 1 minute
    except Exception as e:
        print(f"An error occurred: {e}")
        # Sleep before retrying
        time.sleep(30)  # Sleep for 30 seconds before retrying

An error occurred: Expecting value: line 1 column 1 (char 0)


In [7]:
import os
import pandas as pd

# Directory of images
image_dir = r"C:\Users\User\Desktop\DSO\traffic-prediction\backend\temp"

# CSV file path
csv_file = r"C:\Users\User\Desktop\DSO\traffic-prediction\backend\image-processing\label_filters.csv"

# Get the list of camera IDs from the directory
image_files = os.listdir(image_dir)
camera_ids_directory = [filename.split("_")[0] for filename in image_files]

# Read the CSV file
df = pd.read_csv(csv_file)

# Get the list of camera IDs from the CSV file
df["Camera_Id"] = df["Camera_Id"].astype(str)
camera_ids_csv = df["Camera_Id"].tolist()

# Find the camera IDs that are present in the directory but not in the CSV file
camera_ids_not_in_csv = list(set(camera_ids_directory) - set(camera_ids_csv))

# Print the camera IDs
print(camera_ids_not_in_csv)


['1112', '1113', '1111']


In [2]:
from ultralytics import YOLO
import os
import torch
from PIL import Image
import numpy as np
from glob import glob
from tqdm import tqdm
from ultralytics.data.augment import LetterBox
import cv2

In [3]:
model = YOLO('yolov8x.pt')
model.to('cuda')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_r

In [8]:
images_folder = r"C:\Users\User\Desktop\DSO\traffic-prediction\backend\assets4"
output_folder = r"C:\Users\User\Desktop\DSO\traffic-prediction\backend\assets4_labels"
classes_file = r"C:\Users\User\Desktop\DSO\traffic-prediction\backend\assets4_labels\classes.txt"

In [9]:
with open(classes_file, "r") as f:
    class_names_to_keep = f.read().splitlines()
class_id_mapping = {class_name: i for i, class_name in enumerate(class_names_to_keep)}

In [10]:
class_id_mapping

{'car': 0, 'motorcycle': 1, 'bus': 2, 'truck': 3}

In [11]:
# Define confidence threshold
conf_threshold = 0.6

letterbox = LetterBox()

# Iterate over images in the folder
image_paths = glob(os.path.join(images_folder, "*.jpg")) + glob(os.path.join(images_folder, "*.jpeg")) + glob(os.path.join(images_folder, "*.png"))

for image_path in tqdm(image_paths, desc="Processing images"):
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    output_path = os.path.join(output_folder, f"{image_name}.txt")
    print(output_path)

    # Load image
    img = cv2.imread(image_path)

    # Resize and pad image
    img = letterbox(image=img)

    # Inference
    results = model(img)
    classes = results[0].names
    for result in tqdm(results, desc="Processing results"):
        # Get detected labels
        labels = []
        confidences = result.boxes.conf.cpu().numpy()
        boxes = result.boxes.xywhn.cpu().numpy()
        class_names = result.names

        for i in range(len(boxes)):
            label = class_names[result.boxes.cls[i].item()]
            confidence = result.boxes.conf[i]
            box = boxes[i]
            if label in class_names_to_keep and confidence > conf_threshold:
                class_id = class_id_mapping[label]  # Get the integer ID for the class name
                label_str = f"{class_id} {box[0]} {box[1]} {box[2]} {box[3]}"
                labels.append(label_str)
        if labels == []:
            continue

        # Save labels to text file
        with open(output_path, "w") as f:
            f.write("\n".join(labels))

print("Labels saved successfully.")

Processing images:   0%|          | 0/810 [00:00<?, ?it/s]

C:\Users\User\Desktop\DSO\traffic-prediction\backend\assets4_labels\1001_1403_20240127141006_521019.txt

0: 640x640 1 clock, 27.0ms
Speed: 15.0ms preprocess, 27.0ms inference, 410.0ms postprocess per image at shape (1, 3, 640, 640)


Processing images:   0%|          | 1/810 [00:02<27:18,  2.03s/it]

C:\Users\User\Desktop\DSO\traffic-prediction\backend\assets4_labels\1001_1413_20240127141504_495410.txt

0: 640x640 1 car, 1 clock, 27.9ms
Speed: 4.0ms preprocess, 27.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


Processing results: 100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]

...

Processing results: 100%|██████████| 1/1 [00:00<00:00, 499.86it/s]

C:\Users\User\Desktop\DSO\traffic-prediction\backend\assets4_labels\9706_1448_20240127145000_501015.txt



0: 640x640 1 person, 16 cars, 1 motorcycle, 7 trucks, 1 traffic light, 1 clock, 26.3ms
Speed: 2.3ms preprocess, 26.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


Processing images: 100%|██████████| 810/810 [00:43<00:00, 18.58it/s]

Labels saved successfully.


In [4]:
import pandas as pd
traffic_stats = pd.read_csv(r'C:\Users\User\Desktop\DSO\traffic-prediction\traffic_stats.csv')
jam_pred = []
jam_pred.extend(traffic_stats[(traffic_stats['Direction'] == "KPE (TPE)") & (
                traffic_stats['Camera_Id'] == int(1001))]['Jam'].values)

In [5]:
jam_pred

[0, 0, 0, 0]